In [0]:
import os
import random

from skimage import io
import cv2
import numpy as np
import PIL
from PIL import Image, ImageOps

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls "/content/drive/My Drive/BDMA/BDRP_clone"

generated_images.zip  images.zip


In [0]:
!cp 'drive/My Drive/BDMA/BDRP_clone/images.zip' .
!unzip images.zip
!mkdir images
!mv *.jpeg images/
!mkdir generated_images

In [0]:
shapes = {}
shoes = os.listdir('images')
for shoe in shoes:
  img = io.imread('images/' + shoe)
  if img.shape not in shapes:
    shapes[img.shape] = 0
  shapes[img.shape] += 1

  if img.shape != (592,592,3):
    os.remove('images/'+shoe)

print(shapes)

{(592, 592, 3): 667, (540, 540, 3): 465, (30, 75, 3): 211, (593, 444, 3): 362, (780, 780, 3): 182, (28, 70, 3): 2, (540, 540): 6, (790, 632, 3): 1, (592, 592): 2, (780, 780): 2, (960, 960, 3): 1, (2560, 2560, 3): 1, (498, 373, 3): 1, (620, 620, 3): 1, (580, 580, 3): 1}


In [0]:
class ShoesDataset(Dataset):
  def __init__(self, images, transform=None):
    self.images = images
    self.transform = transform
  
  def __len__(self, ):
    return len(self.images)
  
  def __getitem__(self, index):
    image = self.images[index]

    if self.transform:
      image = self.transform(image)
    # swap color axis because
    # numpy image: H x W x C
    # torch image: C X H X W
    return image

In [0]:
images = []

for image in os.listdir('images'):
  images.append(Image.open('images/'+image))

In [0]:
imageSize = 64
# torch.from_numpy(image.transpose((2, 0, 1)))

transform = transforms.Compose([transforms.Resize(imageSize),
                                transforms.CenterCrop(imageSize),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                               ])

shoes_dataset = ShoesDataset(images, transform)
#img = shoes_dataset[0]
#img.view(img.size(0), -1).size()

In [0]:
batchSize = 64
nz = 100
ngf = 64
ndf = 64
niter = 10000
lr = 0.0002
beta1 = 0.5
outf = 'generated_images'
nc = 3
device = 'cuda:0'

In [0]:
dataloader = torch.utils.data.DataLoader(shoes_dataset, batch_size=batchSize, shuffle=True)

In [0]:
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [0]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.ngpu = 1
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )

    def forward(self, input):
        if input.is_cuda and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(input)
        return output

In [0]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.ngpu = 1
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        if input.is_cuda and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(input)

        return output.view(-1, 1).squeeze(1)

In [0]:
netG = Generator().to(device)
netG.apply(weights_init)

Generator(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Dropout(p=0.1, inplace=False)
    (3): ReLU(inplace=True)
    (4): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (5): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Dropout(p=0.1, inplace=False)
    (7): ReLU(inplace=True)
    (8): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): Dropout(p=0.1, inplace=False)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): Dropout(p=0.1, inp

In [0]:
netD = Discriminator().to(device)
netD.apply(weights_init)

Discriminator(
  (main): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): Dropout(p=0.1, inplace=False)
    (2): LeakyReLU(negative_slope=0.2, inplace=True)
    (3): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): Dropout(p=0.1, inplace=False)
    (6): LeakyReLU(negative_slope=0.2, inplace=True)
    (7): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (8): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): Dropout(p=0.1, inplace=False)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (12): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): Dropout(p=0.1, inplace=False)
    (14): LeakyReLU(ne

In [0]:
criterion = nn.BCELoss()

fixed_noise = torch.randn(batchSize, nz, 1, 1, device=device)
real_label = 1
fake_label = 0

# setup optimizer
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

In [0]:
netD.train()
netG.train()

for epoch in range(niter):
    for i, data in enumerate(dataloader):
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        # train with real

        # Add noise to the real data
        variance = 1 / (epoch + 0.000001)
        init_noise = torch.distributions.Normal(torch.tensor([0.0]), torch.tensor([variance]))
        data.add_(init_noise.sample(data.shape).reshape(data.shape))


        netD.zero_grad()
        real_cpu = data.to(device)
        batch_size = real_cpu.size(0)
        label = torch.full((batch_size,), real_label, device=device)

        output = netD(real_cpu)
        errD_real = criterion(output, label)
        errD_real.backward()
        D_x = output.mean().item()

        # train with fake
        noise = torch.randn(batch_size, nz, 1, 1, device=device)
        fake = netG(noise)
        label.fill_(fake_label)
        output = netD(fake.detach())
        errD_fake = criterion(output, label)
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        errD = errD_real + errD_fake
        torch.nn.utils.clip_grad_value_(netD.parameters(), 5.)  # clip gradient if its norm exceed 5
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        label.fill_(real_label)  # fake labels are real for generator cost
        output = netD(fake)
        errG = criterion(output, label)
        errG.backward()
        torch.nn.utils.clip_grad_value_(netG.parameters(), 5.)  # clip gradient if its norm exceed 5
        D_G_z2 = output.mean().item()
        optimizerG.step()

        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f'
              % (epoch, niter, i, len(dataloader),
                 errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))
        if i % 100 == 0:
            vutils.save_image(real_cpu,
                    '%s/real_samples.png' % outf,
                    normalize=True)
            fake = netG(fixed_noise)
            vutils.save_image(fake.detach(),
                    '%s/fake_samples_epoch_%03d.png' % (outf, epoch),
                    normalize=True)

In [0]:
normal = torch.distributions.Normal(torch.tensor([0.0]), torch.tensor([1.0]))
normal.sample((1,)).reshape((1,)).shape

torch.Size([1])

In [0]:
torch.save(netD.state_dict(), 'discriminator_400.weights')
torch.save(netG.state_dict(), 'generator_400.weights')

In [0]:
netD.load_state_dict(torch.load('discriminator_400.weights'))

<All keys matched successfully>

In [0]:
for name, param in netD.named_parameters():
  print(name, param.shape)

main.1.weight torch.Size([64, 3, 4, 4])
main.4.weight torch.Size([128, 64, 4, 4])
main.5.weight torch.Size([128])
main.5.bias torch.Size([128])
main.8.weight torch.Size([256, 128, 4, 4])
main.9.weight torch.Size([256])
main.9.bias torch.Size([256])
main.12.weight torch.Size([512, 256, 4, 4])
main.13.weight torch.Size([512])
main.13.bias torch.Size([512])
main.16.weight torch.Size([1, 512, 4, 4])


In [0]:
netG.load_state_dict(torch.load('generator_400.weights'))

<All keys matched successfully>

In [0]:
netG.eval()
noise = torch.randn(batch_size, nz, 1, 1, device=device)
fake = netG(noise)
vutils.save_image(fake.detach(),
                    '%s/fake_samples.png' % (outf),
                    normalize=True)

In [0]:
!zip -r generated_images.zip generated_images/

In [0]:
!cp 'generated_images.zip' 'drive/My Drive/BDRP_clone'

PRO Gan

In [0]:
!pip install pro-gan-pth

  Created wheel for pro-gan-pth: filename=pro_gan_pth-2.1.1-cp36-none-any.whl size=15112 sha256=560a07cbda28917e3a9326c6e8ccb9fe134659c9b61b3687f29ccbf775ddc5f0
  Stored in directory: /root/.cache/pip/wheels/e2/0c/b5/1581b3f4df4acb1d3c2f5335cf1a0177d0058e6ff9595774cc
Successfully built pro-gan-pth


In [0]:
import pro_gan_pytorch.PRO_GAN as pg
import torchvision as tv

In [0]:
# some parameters:
depth = 4
# hyper-parameters per depth (resolution)
num_epochs = [10, 20, 20, 20]
fade_ins = [50, 50, 50, 50]
batch_sizes = [128, 128, 128, 128]
latent_size = 128

In [0]:
pro_gan = pg.ProGAN(depth=depth, latent_size=latent_size, device=device)

In [0]:
pro_gan.train(
    dataset=shoes_dataset,
    epochs=num_epochs,
    fade_in_percentage=fade_ins,
    batch_sizes=batch_sizes
)

Starting the training process ... 


Currently working on Depth:  0
Current resolution: 4 x 4

Epoch: 1


RuntimeError: ignored